# Electroprophet⚡️

In this notebook, I will do the first steps on our final project for Le Wagon's Data Science bootcamp. Here, I'll try to model the carbon emission in France using weather data, in order to provide recomendations for users to reduce their carbon footprint 👣.

## Getting the data

In [75]:
%load_ext autoreload
%autoreload 2

### 1. Wheater API

Here we'll be able to get weather data of a chosen city. 

As the energy data is given by region, later we'll have to check how many cities we have to take in consideration in order to get the weather in a given region.

In [87]:
def get_weather(city, years=10, overwrite=False):
    
    '''
    This function receives the name of a city and a number of years, and returns a dataframe 
    with weather data from this city during those past years
    '''
    
    import requests
    import datetime
    from dateutil.relativedelta import relativedelta
    import pandas as pd
    import os.path
    
    path = './raw_data/df_' + city.lower() + '_weather.csv'
    file_exists = os.path.isfile(path) 
    
    if file_exists and not overwrite:
        
        print('Found a file for this city. Importing...')
        
        weather_df = pd.read_csv(path, index_col=0)
        
    else:
        
        print('Creating a new .csv file for this city')
        
        # First we declare the weather parameters. Here we'll be taking all params supported by the API
        weather_params = ['temperature_2m','relativehumidity_2m','dewpoint_2m',
                      'apparent_temperature','pressure_msl','surface_pressure',
                      'precipitation','rain','snowfall','cloudcover',
                      'cloudcover_low','cloudcover_mid','cloudcover_high',
                      'shortwave_radiation','direct_radiation','direct_normal_irradiance',
                      'diffuse_radiation','windspeed_10m','windspeed_100m',
                      'winddirection_10m','winddirection_100m','windgusts_10m',
                      'et0_fao_evapotranspiration','weathercode','vapor_pressure_deficit',
                      'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm',
                      'soil_temperature_28_to_100cm','soil_temperature_100_to_255cm',
                      'soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                      'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm']

        # This request is done in order to get the latitude and longitude of the desired city
        city_response = requests.get('https://geocoding-api.open-meteo.com/v1/search',
                           params = {'name': city}).json()

        lat = city_response['results'][0]['latitude']
        lon = city_response['results'][0]['longitude']

        # Then we compute the dates used to get the weather data
        ## The API only has data until 9 days ago
        end_date = (datetime.date.today() - relativedelta(days=9)).strftime('%Y-%m-%d') 
        start_date = (datetime.date.today() - relativedelta(years=years)).strftime('%Y-%m-%d')

        # So we make the request to the weather API archive
        weather_response = requests.get('https://archive-api.open-meteo.com/v1/archive',
                           params = {'latitude': lat,
                                    'longitude': lon,
                                    'start_date': start_date,
                                    'end_date': end_date,
                                    'hourly': weather_params}).json()

        weather_df = pd.DataFrame(weather_response['hourly'], columns = ['time'] + weather_params)
        
        weather_df.to_csv(path, index=False)
    
    print('Done')
    return weather_df

In [90]:
from prophecy import data

data.get_weather('Amiens')

Found a file for this city. Importing...
Done ✅


,time,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,pressure_msl,surface_pressure,precipitation,rain,snowfall,...,weathercode,vapor_pressure_deficit,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
0,2013-03-09T00:00,8.5,97,8.1,5.8,1000.3,987.3,0.0,0.0,0.0,...,3,0.03,7.8,7.5,5.3,5.6,0.397,0.374,0.384,0.391
1,2013-03-09T01:00,8.5,97,8.1,6.2,1000.8,987.8,0.0,0.0,0.0,...,3,0.03,7.8,7.5,5.3,5.6,0.396,0.374,0.384,0.391
2,2013-03-09T02:00,8.5,97,8.0,6.6,1001.0,988.0,0.0,0.0,0.0,...,3,0.04,7.7,7.5,5.3,5.6,0.394,0.375,0.384,0.391
3,2013-03-09T03:00,8.4,97,7.9,6.7,1000.9,987.9,0.0,0.0,0.0,...,3,0.04,7.6,7.4,5.3,5.6,0.392,0.375,0.384,0.391
4,2013-03-09T04:00,8.1,97,7.7,6.7,1000.8,987.8,0.0,0.0,0.0,...,3,0.03,7.5,7.4,5.3,5.6,0.391,0.376,0.384,0.391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87451,2023-02-28T19:00,1.9,53,-6.6,-3.5,1029.4,1015.7,0.0,0.0,0.0,...,0,0.33,2.7,3.9,6.4,8.0,0.317,0.328,0.362,0.375
87452,2023-02-28T20:00,1.0,58,-6.5,-3.9,1029.6,1015.9,0.0,0.0,0.0,...,0,0.28,2.1,3.9,6.4,8.0,0.317,0.328,0.361,0.375
87453,2023-02-28T21:00,0.2,61,-6.5,-4.2,1029.9,1016.1,0.0,0.0,0.0,...,0,0.24,1.5,3.8,6.4,8.0,0.317,0.328,0.361,0.375
87454,2023-02-28T22:00,-0.3,63,-6.5,-4.9,1030.3,1016.5,0.0,0.0,0.0,...,0,0.22,1.1,3.6,6.4,8.0,0.317,0.328,0.361,0.375


### 2. Energy API